In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import sqrtm
from scipy.sparse import csgraph

In [2]:
N_TRIAL = 30000
N_ARMS = 100
N_FEATURE = 3
M = 4
np.random.seed(123)

In [3]:
W = np.genfromtxt('W.csv',delimiter=',')

In [4]:
# Create the Adj matrix
def constructAdjMatrix(W, n, threshold): #m
    Adj_mat = np.zeros(shape = (n, n))
    for ui in range(n):
        for uj in range(n):
            Adj_mat[ui][uj] = W[ui][uj]
        # find out the top M similar users in G -> find a threshold
#         if m>0 and m<n:
#             similarity = sorted(W[ui], reverse=True)
#             threshold = similarity[m]
        # trim the graph
            for i in range(n):
                if W[ui][i] <= threshold:
                    Adj_mat[ui][i] = 0;
        Adj_mat[ui] /= sum(Adj_mat[ui])
    return Adj_mat

# Create the laplacian matrix
def constructLaplacianMatrix(W, n, Gepsilon):
    G = W.copy()
    #Convert adjacency matrix of weighted graph to adjacency matrix of unweighted graph
    for i in range(n):
        for j in range(n):
            if G[i][j] > 0:
                G[i][j] = 1
    L = csgraph.laplacian(G, normed = False)
    I = np.identity(n = G.shape[0])
    GW = I + Gepsilon*L  # W is a double stochastic matrix
    return GW.T

In [5]:
# Create the F matrix
lda = 1
threshold = 0.5
test_adj = constructAdjMatrix(W, N_ARMS, threshold)
test_F = constructLaplacianMatrix(test_adj, N_ARMS, lda)
F_kron = np.kron(test_F, np.identity(n=N_FEATURE))
FInv_Init = sqrtm(np.linalg.inv(np.kron(test_F, np.identity(n=N_FEATURE))))

In [6]:
FInv_Init

array([[ 1.93378283e-01+2.73691106e-48j, -6.72697549e-17-7.22223729e-35j,
        -3.99702502e-17+1.20370622e-34j, ...,
         1.44820277e-03+0.00000000e+00j, -2.07239911e-17+1.20370622e-35j,
        -2.51963704e-17-1.80555932e-35j],
       [-1.50018683e-17-4.37905770e-47j,  1.93378283e-01-5.42101086e-20j,
        -9.54097912e-18-1.89735380e-19j, ...,
        -3.74700272e-17-2.69630192e-33j,  1.44820277e-03+3.85185989e-33j,
         4.42601001e-17-1.15555797e-33j],
       [-2.24546128e-17+0.00000000e+00j, -1.49619900e-17+8.13151629e-20j,
         1.93378283e-01+2.16840434e-19j, ...,
         1.65654353e-17+6.16297582e-33j,  1.02795551e-17+1.23259516e-32j,
         1.44820277e-03-1.09476443e-47j],
       ...,
       [ 1.44820277e-03-5.47382213e-48j, -3.06768504e-17-7.22223729e-35j,
         1.62139200e-17-2.18952885e-47j, ...,
         1.95829134e-01-4.33680869e-19j, -1.73472348e-17-2.73691106e-48j,
        -3.25260652e-19+2.16840434e-19j],
       [-2.95434795e-17+3.42113883e-49j,  1.